In [3]:
import time
import Jetson.GPIO as GPIO

# # GPIO 핀 설정
# PIN = 8  # 센서의 LOW 신호를 연결할 GPIO 핀 번호 (BCM 기준)

# # GPIO 초기화
# GPIO.setmode(GPIO.BCM)
# GPIO.setup(PIN, GPIO.IN)

def get_pm25_concentration(sample_time_ms=3000):
    PIN = 8  # 센서의 LOW 신호를 연결할 GPIO 핀 번호 (BCM 기준)

    # GPIO 초기화
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(PIN, GPIO.IN)
    """
    GPIO 핀에서 LOW 신호를 측정하여 PM2.5 농도를 계산합니다.

    :param sample_time_ms: 샘플링 시간 (기본값: 30000ms = 30초)
    :return: PM2.5 농도 (ug/m3)
    """
    lowpulseoccupancy = 0  # LOW 신호 지속 시간 누적 (마이크로초 단위)
    start_time = time.time() * 1000  # 현재 시간 (밀리초)

    # print("미세먼지 농도 측정 중...")

    while (time.time() * 1000 - start_time) < sample_time_ms:
        if GPIO.input(PIN) == 0:  # LOW 신호 감지
            pulse_start = time.time()
            while GPIO.input(PIN) == 0:  # LOW 상태 지속
                pass
            pulse_end = time.time()
            duration = (pulse_end - pulse_start) * 1_000_000  # 마이크로초로 변환
            lowpulseoccupancy += duration

    # Ratio 및 농도 계산
    ratio = lowpulseoccupancy / (sample_time_ms * 10.0)
    concentration = 1.1 * 0.4 * (ratio ** 3) - 3.8 * 0.4 *  (ratio ** 2) + 520 * 0.4 * ratio + 0.62  # ug/m3 단위
    return str(concentration)

try:
    pm25_concentration = get_pm25_concentration()
    print(f"PM2.5 농도: {float(pm25_concentration):.2f} ug/m3")
except KeyboardInterrupt:
    print("\n측정을 중단했습니다.")
finally:
    GPIO.cleanup()  # GPIO 핀 초기화


/usr/lib/python3/dist-packages/Jetson/GPIO/gpio.py:386: RuntimeWarning: This channel is already in use, continuing anyway. Use GPIO.setwarnings(False) to disable warnings
  RuntimeWarning)


PM2.5 농도: 0.62 ug/m3
